### Nutritional information from openfoodfacts

In [1]:
import os
import pandas as pd
import numpy as np
import openfoodfacts
from sqlalchemy import create_engine, text
from dotenv import load_dotenv, find_dotenv

In [2]:
load_dotenv(find_dotenv())

SQLALCHEMY_DATABASE_URL = os.environ["cockroack_db_deta_connect_string"]

engine = create_engine(SQLALCHEMY_DATABASE_URL)

Python-dotenv could not parse statement starting at line 10
Python-dotenv could not parse statement starting at line 11
Python-dotenv could not parse statement starting at line 15
Python-dotenv could not parse statement starting at line 16


In [3]:
barcodes = pd.read_csv("Scan session.csv", header=None).T.to_numpy().tolist()[0]
np.savetxt("food_barcodes.csv", barcodes + [5411188512271, 8718403014549], delimiter=",", fmt='%s')
barcodes = pd.read_csv("food_barcodes.csv", header=None).T.to_numpy().tolist()[0]

In [4]:
np.array(barcodes)

array([8714779001053, 8718452427512, 8711852200181, 5411188111252,
       7394376616419, 8718452265732, 8718452459698, 5202390019319,
       8714779003187, 8718452451043, 8719324157025, 8710624281052,
       8710624280499, 8710624280437, 5060072970019, 8711299021738,
       8711299021776, 8711299021721, 8711299021714,      50818853,
       8718452334827, 8718452261697, 8713965500189, 8718452375783,
       8710398517388, 8690661080330, 8719153025236, 8718452396535,
       8718452469352, 8719189108200, 8715700131207, 8718452469352,
       8718452375783, 8713965500189, 8690661080330, 8718452459698,
       8718452459698, 5411188111078, 8719153025236, 8710398517388,
       8718692781863, 8710861912900, 5411188512271, 8718403014549])

In [5]:
all_foods_df = pd.DataFrame()
failed_barcodes = list()
for barcode_int in barcodes:
    barcode=str(barcode_int)
    try:
        search_result = openfoodfacts.products.get_product(barcode)
        food_df = pd.concat([pd.DataFrame({"product_name":search_result["product"]["product_name"],
                                          "barcode":barcode}, index=[0]), 
                         pd.DataFrame(search_result["product"]["nutriments"], index=[0])], axis=1)
        all_foods_df = pd.concat([all_foods_df, food_df])
    except:
        print(f"Barcode == {barcode} not available in the database")
        failed_barcodes.append(barcode_int)
_ = all_foods_df.reset_index(inplace=True, drop=True)

Barcode == 8718452396535 not available in the database
Barcode == 8719189108200 not available in the database


In [22]:
foods_subset_df = (all_foods_df
.filter(regex="product_name|barcode|100g")
.drop("energy_100g", axis=1))
foods_subset_df.columns=foods_subset_df.columns.str.replace("-", "_")

In [24]:
foods_subset_df.head()

,product_name,barcode,carbohydrates_100g,energy_kcal_100g,fat_100g,fruits_vegetables_nuts_estimate_from_ingredients_100g,nutrition_score_fr_100g,proteins_100g,salt_100g,saturated_fat_100g,...,biotin_100g,magnesium_100g,phosphorus_100g,vitamin_b9_100g,vitamin_e_100g,vitamin_pp_100g,calcium_100g,vitamin_b12_100g,vitamin_b2_100g,vitamin_d_100g
0,Falafel,8714779001053,24.8,232,9.7,0.0,3.0,8.4,1.20,0.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jumbo zoete dip van paprika walnoten en couscous,8718452427512,15.0,207,14.4,NaN,NaN,3.2,NaN,1.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hummus pikant,8711852200181,12.4,256,19.1,NaN,NaN,6.2,1.34,1.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Soja lait,5411188111252,2.5,39,1.8,NaN,0.0,0.8,0.08,0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Haferdrink Orange Mango,7394376616419,11.0,54,0.5,10.0,1.0,1.0,0.01,0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# quick print statement check
# with engine.connect() as conn:
#     result = conn.execute(text("SELECT * FROM nutritional_information"))
#     print(result.all())

In [25]:
# upload data frame to SQL (be aware of the if_exists)
with engine.connect() as conn:
    foods_subset_df.to_sql('nutritional_information', 
    con=conn, 
    if_exists='append')

In [16]:
# drop table syntax
# with engine.connect() as conn:
#     result = conn.execute(text('DROP TABLE IF EXISTS nutritional_information;'))

In [26]:
nut_info_df = pd.read_sql_table("nutritional_information", 
                                con=engine, index_col="index")
display(nut_info_df.head())
nut_info_df.info()

,product_name,barcode,carbohydrates_100g,energy_kcal_100g,fat_100g,fruits_vegetables_nuts_estimate_from_ingredients_100g,nutrition_score_fr_100g,proteins_100g,salt_100g,saturated_fat_100g,...,biotin_100g,magnesium_100g,phosphorus_100g,vitamin_b9_100g,vitamin_e_100g,vitamin_pp_100g,calcium_100g,vitamin_b12_100g,vitamin_b2_100g,vitamin_d_100g
index,,,,,,,,,,,,,,,,,,,,,
0,Falafel,8714779001053,24.8,232,9.7,0.0,3.0,8.4,1.20,0.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Jumbo zoete dip van paprika walnoten en couscous,8718452427512,15.0,207,14.4,NaN,NaN,3.2,NaN,1.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Hummus pikant,8711852200181,12.4,256,19.1,NaN,NaN,6.2,1.34,1.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Soja lait,5411188111252,2.5,39,1.8,NaN,0.0,0.8,0.08,0.3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Haferdrink Orange Mango,7394376616419,11.0,54,0.5,10.0,1.0,1.0,0.01,0.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


<class 'pandas.core.frame.DataFrame'>
Int64Index: 42 entries, 0 to 41
Data columns (total 28 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   product_name                                           42 non-null     object 
 1   barcode                                                42 non-null     object 
 2   carbohydrates_100g                                     42 non-null     float64
 3   energy_kcal_100g                                       42 non-null     int64  
 4   fat_100g                                               42 non-null     float64
 5   fruits_vegetables_nuts_estimate_from_ingredients_100g  15 non-null     float64
 6   nutrition_score_fr_100g                                12 non-null     float64
 7   proteins_100g                                          42 non-null     float64
 8   salt_100g                                           